In [1]:
from tqdm import tqdm
import os
import numpy as np
import time
import random
import pandas as pd
from scipy import sparse
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings("ignore")

# 读取数据集

In [2]:
data_org_dir = 'data/data_tencent/'
data_prep_dir = 'data/preprocess/'
# data_prep_dir = 'data/sample/'
sample_test = False

In [3]:
if sample_test:
    df_data = pd.read_csv(data_prep_dir + 'train_feat_merge_mini.csv')
else:
    df_data = pd.read_csv(data_prep_dir + 'train_feat_merge.csv')

train_idx_lst = list(df_data[df_data['n_parts'] != 1].index)
valid_idx_lst = list(df_data[df_data['n_parts'] == 1].index)


In [4]:
df_data.columns

Index(['n_parts', 'aid', 'uid', 'label', 'LBS', 'age', 'appIdAction',
       'appIdInstall', 'carrier', 'consumptionAbility', 'ct', 'education',
       'gender', 'house', 'interest1', 'interest2', 'interest3', 'interest4',
       'interest5', 'kw1', 'kw2', 'kw3', 'marriageStatus', 'os', 'topic1',
       'topic2', 'topic3', 'advertiserId', 'campaignId', 'creativeId',
       'creativeSize', 'adCategoryId', 'productId', 'productType'],
      dtype='object')

# 缺失率筛选

In [5]:
# 缺失情况统计
total = len(df_data)
for col_name in list(df_data.columns):
    df_missing = (df_data[df_data[col_name] == '-1'])
    
    print(col_name, ':', round(len(df_missing)/total, 2) * 100, '%')

n_parts : 0.0 %
aid : 0.0 %
uid : 0.0 %
label : 0.0 %
LBS : 0.0 %
age : 0.0 %
appIdAction : 98.0 %
appIdInstall : 98.0 %
carrier : 0.0 %
consumptionAbility : 0.0 %
ct : 0.0 %
education : 0.0 %
gender : 0.0 %
house : 0.0 %
interest1 : 9.0 %
interest2 : 34.0 %
interest3 : 97.0 %
interest4 : 98.0 %
interest5 : 25.0 %
kw1 : 10.0 %
kw2 : 3.0 %
kw3 : 95.0 %
marriageStatus : 0.0 %
os : 0.0 %
topic1 : 9.0 %
topic2 : 4.0 %
topic3 : 95.0 %
advertiserId : 0.0 %
campaignId : 0.0 %
creativeId : 0.0 %
creativeSize : 0.0 %
adCategoryId : 0.0 %
productId : 0.0 %
productType : 0.0 %


## 删除特征

1. 删除category特别大特别稀疏特征: 'appIdInstall', 'appIdAction', 'marriageStatus';
2. 删除缺失情况严重特征: 'interest3', 'interest4', 'kw3', 'topic3';


In [6]:
feat2drop = ['appIdInstall', 'appIdAction', 'marriageStatus', 'interest3', 'interest4', 'kw3', 'topic3']
df_data.drop(feat2drop, axis=1, inplace=True)


In [7]:
df_data.columns

Index(['n_parts', 'aid', 'uid', 'label', 'LBS', 'age', 'carrier',
       'consumptionAbility', 'ct', 'education', 'gender', 'house', 'interest1',
       'interest2', 'interest5', 'kw1', 'kw2', 'os', 'topic1', 'topic2',
       'advertiserId', 'campaignId', 'creativeId', 'creativeSize',
       'adCategoryId', 'productId', 'productType'],
      dtype='object')

# 定长离散特征处理
- 用户类: 'LBS', 'age', 'carrier', 'consumptionAbility', 'education', 'gender', 'house', 'os'
- 广告类: 'aid', 'advertiserId', 'campaignId', 'creativeId', 'creativeSize', 'adCategoryId','productId', 'productType'



In [8]:
discrete_feat_lst = ['LBS', 'age', 'carrier', 'consumptionAbility', 'education', 'gender', 'house', 'os',
                     'aid', 'advertiserId', 'campaignId', 'creativeId', 'creativeSize', 'adCategoryId',
                     'productId', 'productType']
t0 = time.time()
for discrete_feat in tqdm(discrete_feat_lst):
    enc = LabelEncoder()
    try:
        df_data[discrete_feat] = enc.fit_transform(df_data[discrete_feat].apply(int))
    except:
        df_data[discrete_feat] = enc.fit_transform(df_data[discrete_feat])


100%|██████████| 16/16 [01:02<00:00,  3.93s/it]


In [9]:
def one_hot_encoding(df_feat2enc):
    """

    :param df_feat2enc:
    :type df_feat2enc:
    :return:
    :rtype:
    """
    
    one_hot_enc.fit(df_feat2enc.values.reshape(-1, 1))
    feat_enc_arr = one_hot_enc.transform(df_feat2enc.values.reshape(-1, 1))#.toarray()

    return feat_enc_arr

# 数据量太大了, 一下子肝不完, 必须分开
discrete_feat_lst1 = ['LBS', 'age', 'carrier', 'consumptionAbility', 'education', 'gender']
discrete_feat_lst2 = ['aid', 'advertiserId', 'campaignId', 'creativeId', 'creativeSize']
discrete_feat_lst3 = ['adCategoryId', 'productId', 'productType', 'house', 'os']

discrete_feat = [discrete_feat_lst1, discrete_feat_lst2, discrete_feat_lst3]
# 初始化
one_hot_enc = OneHotEncoder()

x_train = pd.DataFrame()
x_valid = pd.DataFrame()
###### 数据量很大的时候用这个 ######
for feat_lst in discrete_feat:
    
    for feat in tqdm(feat_lst):
        # 合并训练
        df_feat2enc = df_data[feat]
        one_hot_enc.fit(df_feat2enc.values.reshape(-1, 1))
        # 训练集
        train2enc = df_data.loc[train_idx_lst][feat]
        train_enc_arr = one_hot_enc.transform(train2enc.values.reshape(-1, 1))
        x_train = sparse.hstack([x_train, train_enc_arr])
        # 验证集
        valid2enc = df_data.loc[valid_idx_lst][feat]
        valid_enc_arr = one_hot_enc.transform(valid2enc.values.reshape(-1, 1))
        x_valid = sparse.hstack([x_valid, valid_enc_arr])
    
train_f_dir = data_prep_dir + 'train_x_sparse_onehot.npz'
sparse.save_npz(train_f_dir, x_train)

valid_f_dir = data_prep_dir + 'valid_x_sparse_onehot.npz'
sparse.save_npz(valid_f_dir, x_valid)

print(train_f_dir, 'is saved! Array shape is', x_train.shape)
print(valid_f_dir, 'is saved! Array shape is', x_valid.shape)

###### 测试数据用这个 ######
# train_x = np.empty([len(train_idx_lst), 1], dtype=int)
# valid_x = np.empty([len(valid_idx_lst), 1], dtype=int) 
# for feat in tqdm(discrete_feat_lst):
    
#     # 训练集
#     df_feat2enc_train = df_data.loc[train_idx_lst][feat]
#     train_enc_arr = one_hot_encoding(df_feat2enc_train)
#     train_x = sparse.hstack([train_x, train_enc_arr])
#     # 验证集
#     df_feat2enc_valid = df_data.loc[valid_idx_lst][feat]
#     valid_enc_arr = one_hot_encoding(df_feat2enc_valid)
#     valid_x = sparse.hstack([valid_x, valid_enc_arr])
    
# train_f_dir = data_prep_dir + 'train_x_sparse_onehot.npz'
# sparse.save_npz(train_f_dir, train_enc_arr)

# valid_f_dir = data_prep_dir + 'valid_x_sparse_onehot.npz'
# sparse.save_npz(valid_f_dir, valid_enc_arr)

# print(train_f_dir, 'is saved! Array shape is', train_x.shape)
# print(valid_f_dir, 'is saved! Array shape is', valid_x.shape)




100%|██████████| 5/5 [00:50<00:00, 10.15s/it]


data/preprocess/train_x_sparse_onehot.npz is saved! Array shape is (7038840, 1526)
data/preprocess/valid_x_sparse_onehot.npz is saved! Array shape is (1759974, 1526)


# 序列型特征处理
- 用户类: 'ct', 'interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'os'，'topic1', 'topic2', 

In [10]:
sequence_feat_lst = ['ct', 'interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']

## 构建序列长度特征
len_static_features = [x+'_len' for x in sequence_feat_lst]

In [11]:
# 初始化
x_train = pd.DataFrame()
x_valid = pd.DataFrame()

df_feat2len = df_data[sequence_feat_lst].applymap(lambda x: len(x.split(' ')))

# 构建训练集&验证集
df_train_enc = df_feat2len.loc[train_idx_lst].values
df_valid_enc = df_feat2len.loc[valid_idx_lst].values

x_train = sparse.hstack([x_train, df_train_enc])
x_valid = sparse.hstack([x_valid, df_valid_enc])

train_f_dir = data_prep_dir + 'train_x_sparse_seq_len.npz'
sparse.save_npz(train_f_dir, x_train)

valid_f_dir = data_prep_dir + 'valid_x_sparse_seq_len.npz'
sparse.save_npz(valid_f_dir, x_valid)

print(train_f_dir, 'is saved! Array shape is', x_train.shape)
print(valid_f_dir, 'is saved! Array shape is', x_valid.shape)

data/preprocess/train_x_sparse_seq_len.npz is saved! Array shape is (7038840, 8)
data/preprocess/valid_x_sparse_seq_len.npz is saved! Array shape is (1759974, 8)


In [12]:
df_feat2len.shape

(8798814, 8)

## 构建序列count特征

In [13]:
# 初始化
x_train = pd.DataFrame()
x_valid = pd.DataFrame()
cnt_enc = CountVectorizer()

for sequence_feat in tqdm(sequence_feat_lst[1:]):  # TODO: 'ct'未构建该类特征
    # 合并训练
    df_feat2cnt = df_data[sequence_feat]
    cnt_enc.fit(df_feat2cnt.values)
    # 训练集
    train2enc = df_data.loc[train_idx_lst][sequence_feat]
    train_enc_arr = cnt_enc.transform(train2enc.values)
    x_train = sparse.hstack([x_train, train_enc_arr])
    # 验证集
    valid2enc = df_data.loc[valid_idx_lst][sequence_feat]
    valid_enc_arr = cnt_enc.transform(valid2enc.values)
    x_valid = sparse.hstack([x_valid, valid_enc_arr])
    
train_f_dir = data_prep_dir + 'train_x_sparse_cntv.npz'
sparse.save_npz(train_f_dir, x_train)

valid_f_dir = data_prep_dir + 'valid_x_sparse_cntv.npz'
sparse.save_npz(valid_f_dir, x_valid)

print(train_f_dir, 'is saved! Array shape is', x_train.shape)
print(valid_f_dir, 'is saved! Array shape is', x_valid.shape)


  0%|          | 0/7 [03:33<?, ?it/s]


KeyboardInterrupt: 

# 用户id类特征处理（未处理）
- 用户类: 'uid'
另外，因为也存在量大稀疏的问题，但是比赛的问题就是针对用户对广告的点击，因此uid对模型来说还是很有必要进行建模的，但是uid不使用category的方式建模，而是使用count和转化率建模：
- uid_count : 对uid进行出现频次的建模。
- uid_pos_count : 对uid进行正样本中的出现频次的建模。
- uid_ad_features_pos_count : 对uid组合所有广告特征'ad_static_feature'进行正样本中的出现频次的建模。
- 对uid进行如此多的建模方式是为了能对uid进行更加详细的表述，因为category的每一个特征就是一对一的，而count特征是多对一的，存在大量的信息损失，因此需要进行更多不同角度的建模，才能更好地表述uid。
特征方面还有一些长尾处理，未出现id的统一映射，长度和count特征的未出现次数的取临近值等trick。

# 特征选择
## 加载&合并特征

In [6]:
feat_fname_train = ['train_x_sparse_onehot.npz', 'train_x_sparse_seq_len.npz', 'train_x_sparse_cntv.npz']
feat_fname_valid = ['valid_x_sparse_onehot.npz', 'valid_x_sparse_seq_len.npz', 'valid_x_sparse_cntv.npz']
x_train = pd.DataFrame()
x_valid = pd.DataFrame()
for tr, vl in zip(feat_fname_train, feat_fname_valid):
    train_f = sparse.load_npz(data_prep_dir + tr).tocsr()
    valid_f = sparse.load_npz(data_prep_dir + vl).tocsr()
    
    x_train = sparse.hstack((x_train, train_f)).tocsc()
    x_valid = sparse.hstack((x_valid, valid_f)).tocsc()
    print(tr, train_f.shape)
    print(vl, valid_f.shape)
    
print('x_train:', x_train.shape)
print('x_valid:', x_valid.shape)

print('Loading & merging features completed!')



train_x_sparse_onehot.npz (7038840, 1526)
valid_x_sparse_onehot.npz (1759974, 1526)
train_x_sparse_seq_len.npz (7038840, 8)
valid_x_sparse_seq_len.npz (1759974, 8)
train_x_sparse_cntv.npz (7038840, 323142)
valid_x_sparse_cntv.npz (1759974, 323142)
x_train: (7038840, 324676)
x_valid: (1759974, 324676)
Loading & merging features completed!


In [7]:
# construct y vector
y_train = np.array(df_data[df_data['n_parts'] != 1]['label'])
y_valid = np.array(df_data[df_data['n_parts'] == 1]['label'])

print('y_train:', y_train.shape)
print('y_valid:', y_valid.shape)

y_train: (7038840,)
y_valid: (1759974,)


## LGB选择特征

In [8]:
import pandas as pd
from lightgbm import LGBMClassifier
import time
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')



In [9]:
clf = LGBMClassifier(boosting_type='gbdt',
                     num_leaves=31, max_depth=-1, 
                     learning_rate=0.1, n_estimators=10000, 
                     subsample_for_bin=200000, objective=None,
                     class_weight=None, min_split_gain=0.0, 
                     min_child_weight=0.001,
                     min_child_samples=20, subsample=1.0, subsample_freq=1,
                     colsample_bytree=1.0,
                     reg_alpha=0.0, reg_lambda=0.0, random_state=None,
                     n_jobs=-1, silent=False)


In [10]:
clf.fit(x_train, y_train, eval_set=[(x_train, y_train),(x_valid, y_valid)], 
        eval_names =['train','valid'],
        eval_metric='auc', early_stopping_rounds=100)

[LightGBM] [Info] Number of positive: 337465, number of negative: 6701375
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1232.358202 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 240984
[LightGBM] [Info] Number of data points in the train set: 7038840, number of used features: 120414
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.047943 -> initscore=-2.988606
[LightGBM] [Info] Start training from score -2.988606
[1]	train's auc: 0.633328	train's binary_logloss: 0.189736	valid's auc: 0.632434	valid's binary_logloss: 0.189924
Training until validation scores don't improve for 100 rounds
[2]	train's auc: 0.642899	train's binary_logloss: 0.188124	valid's auc: 0.641948	valid's binary_logloss: 0.18831
[3]	train's auc: 0.651786	train's binary_logloss: 0.186984	valid's auc: 0.65073	valid's binary_logloss: 0.187172
[4]	train's auc: 0.6538

[68]	train's auc: 0.711066	train's binary_logloss: 0.175881	valid's auc: 0.710173	valid's binary_logloss: 0.176176
[69]	train's auc: 0.71138	train's binary_logloss: 0.175826	valid's auc: 0.710459	valid's binary_logloss: 0.176124
[70]	train's auc: 0.711542	train's binary_logloss: 0.175784	valid's auc: 0.710615	valid's binary_logloss: 0.176086
[71]	train's auc: 0.711775	train's binary_logloss: 0.175744	valid's auc: 0.710852	valid's binary_logloss: 0.176045
[72]	train's auc: 0.712111	train's binary_logloss: 0.175693	valid's auc: 0.71116	valid's binary_logloss: 0.176
[73]	train's auc: 0.712392	train's binary_logloss: 0.175644	valid's auc: 0.711463	valid's binary_logloss: 0.175952
[74]	train's auc: 0.712606	train's binary_logloss: 0.175607	valid's auc: 0.711684	valid's binary_logloss: 0.17592
[75]	train's auc: 0.712826	train's binary_logloss: 0.175564	valid's auc: 0.711878	valid's binary_logloss: 0.17588
[76]	train's auc: 0.713025	train's binary_logloss: 0.175526	valid's auc: 0.712083	valid

[140]	train's auc: 0.724414	train's binary_logloss: 0.173507	valid's auc: 0.722513	valid's binary_logloss: 0.173975
[141]	train's auc: 0.72449	train's binary_logloss: 0.173488	valid's auc: 0.722587	valid's binary_logloss: 0.173957
[142]	train's auc: 0.724685	train's binary_logloss: 0.173466	valid's auc: 0.722752	valid's binary_logloss: 0.173938
[143]	train's auc: 0.724803	train's binary_logloss: 0.173444	valid's auc: 0.722849	valid's binary_logloss: 0.17392
[144]	train's auc: 0.724926	train's binary_logloss: 0.173426	valid's auc: 0.722954	valid's binary_logloss: 0.173903
[145]	train's auc: 0.72506	train's binary_logloss: 0.173403	valid's auc: 0.723069	valid's binary_logloss: 0.173883
[146]	train's auc: 0.725153	train's binary_logloss: 0.173383	valid's auc: 0.723134	valid's binary_logloss: 0.173866
[147]	train's auc: 0.725216	train's binary_logloss: 0.173365	valid's auc: 0.723204	valid's binary_logloss: 0.173849
[148]	train's auc: 0.725311	train's binary_logloss: 0.173346	valid's auc: 0

[211]	train's auc: 0.731072	train's binary_logloss: 0.172304	valid's auc: 0.727713	valid's binary_logloss: 0.173015
[212]	train's auc: 0.731153	train's binary_logloss: 0.172291	valid's auc: 0.727791	valid's binary_logloss: 0.173006
[213]	train's auc: 0.731209	train's binary_logloss: 0.172281	valid's auc: 0.727826	valid's binary_logloss: 0.172997
[214]	train's auc: 0.731292	train's binary_logloss: 0.172269	valid's auc: 0.727863	valid's binary_logloss: 0.172991
[215]	train's auc: 0.731367	train's binary_logloss: 0.172259	valid's auc: 0.727886	valid's binary_logloss: 0.172986
[216]	train's auc: 0.731428	train's binary_logloss: 0.172248	valid's auc: 0.727928	valid's binary_logloss: 0.172981
[217]	train's auc: 0.731497	train's binary_logloss: 0.172235	valid's auc: 0.72798	valid's binary_logloss: 0.172972
[218]	train's auc: 0.731575	train's binary_logloss: 0.17222	valid's auc: 0.728033	valid's binary_logloss: 0.172961
[219]	train's auc: 0.731657	train's binary_logloss: 0.172207	valid's auc: 

[282]	train's auc: 0.735903	train's binary_logloss: 0.171463	valid's auc: 0.730484	valid's binary_logloss: 0.172498
[283]	train's auc: 0.735951	train's binary_logloss: 0.171453	valid's auc: 0.730493	valid's binary_logloss: 0.172494
[284]	train's auc: 0.736058	train's binary_logloss: 0.171438	valid's auc: 0.730564	valid's binary_logloss: 0.172484
[285]	train's auc: 0.73613	train's binary_logloss: 0.171429	valid's auc: 0.730573	valid's binary_logloss: 0.172483
[286]	train's auc: 0.736219	train's binary_logloss: 0.171419	valid's auc: 0.730594	valid's binary_logloss: 0.172479
[287]	train's auc: 0.736277	train's binary_logloss: 0.17141	valid's auc: 0.73062	valid's binary_logloss: 0.172475
[288]	train's auc: 0.736338	train's binary_logloss: 0.1714	valid's auc: 0.730644	valid's binary_logloss: 0.172469
[289]	train's auc: 0.736383	train's binary_logloss: 0.171392	valid's auc: 0.730666	valid's binary_logloss: 0.172465
[290]	train's auc: 0.736429	train's binary_logloss: 0.171383	valid's auc: 0.7

...

[1986]	train's auc: 0.793082	train's binary_logloss: 0.161874	valid's auc: 0.737578	valid's binary_logloss: 0.170948
[1987]	train's auc: 0.79314	train's binary_logloss: 0.161862	valid's auc: 0.737627	valid's binary_logloss: 0.170941
[1988]	train's auc: 0.793163	train's binary_logloss: 0.161858	valid's auc: 0.737629	valid's binary_logloss: 0.170941
[1989]	train's auc: 0.793182	train's binary_logloss: 0.161854	valid's auc: 0.737622	valid's binary_logloss: 0.17094
[1990]	train's auc: 0.793215	train's binary_logloss: 0.16185	valid's auc: 0.737625	valid's binary_logloss: 0.170939
[1991]	train's auc: 0.793235	train's binary_logloss: 0.161845	valid's auc: 0.737624	valid's binary_logloss: 0.170939
[1992]	train's auc: 0.793274	train's binary_logloss: 0.161839	valid's auc: 0.737619	valid's binary_logloss: 0.17094
[1993]	train's auc: 0.793295	train's binary_logloss: 0.161835	valid's auc: 0.737614	valid's binary_logloss: 0.17094
[1994]	train's auc: 0.793331	train's binary_logloss: 0.161831	valid's

[2057]	train's auc: 0.79492	train's binary_logloss: 0.16156	valid's auc: 0.737633	valid's binary_logloss: 0.170934
[2058]	train's auc: 0.794943	train's binary_logloss: 0.161557	valid's auc: 0.73763	valid's binary_logloss: 0.170934
[2059]	train's auc: 0.794965	train's binary_logloss: 0.161553	valid's auc: 0.737626	valid's binary_logloss: 0.170935
[2060]	train's auc: 0.794982	train's binary_logloss: 0.16155	valid's auc: 0.737626	valid's binary_logloss: 0.170935
[2061]	train's auc: 0.795005	train's binary_logloss: 0.161546	valid's auc: 0.737632	valid's binary_logloss: 0.170934
[2062]	train's auc: 0.795025	train's binary_logloss: 0.161542	valid's auc: 0.737635	valid's binary_logloss: 0.170934
[2063]	train's auc: 0.795045	train's binary_logloss: 0.161538	valid's auc: 0.737639	valid's binary_logloss: 0.170934
[2064]	train's auc: 0.79509	train's binary_logloss: 0.161526	valid's auc: 0.737705	valid's binary_logloss: 0.170926
[2065]	train's auc: 0.79514	train's binary_logloss: 0.161518	valid's 

[2128]	train's auc: 0.796576	train's binary_logloss: 0.161247	valid's auc: 0.737724	valid's binary_logloss: 0.17091
[2129]	train's auc: 0.796592	train's binary_logloss: 0.161242	valid's auc: 0.737723	valid's binary_logloss: 0.17091
[2130]	train's auc: 0.796625	train's binary_logloss: 0.161237	valid's auc: 0.737727	valid's binary_logloss: 0.17091
[2131]	train's auc: 0.796634	train's binary_logloss: 0.161234	valid's auc: 0.737728	valid's binary_logloss: 0.17091
[2132]	train's auc: 0.796653	train's binary_logloss: 0.16123	valid's auc: 0.737723	valid's binary_logloss: 0.17091
[2133]	train's auc: 0.796672	train's binary_logloss: 0.161226	valid's auc: 0.737724	valid's binary_logloss: 0.17091
[2134]	train's auc: 0.796697	train's binary_logloss: 0.161222	valid's auc: 0.737726	valid's binary_logloss: 0.17091
[2135]	train's auc: 0.796728	train's binary_logloss: 0.161216	valid's auc: 0.737728	valid's binary_logloss: 0.170911
[2136]	train's auc: 0.79676	train's binary_logloss: 0.161212	valid's auc

[2199]	train's auc: 0.79826	train's binary_logloss: 0.160933	valid's auc: 0.737745	valid's binary_logloss: 0.170898
[2200]	train's auc: 0.79828	train's binary_logloss: 0.160929	valid's auc: 0.73775	valid's binary_logloss: 0.170897
[2201]	train's auc: 0.798301	train's binary_logloss: 0.160926	valid's auc: 0.737747	valid's binary_logloss: 0.170897
[2202]	train's auc: 0.798326	train's binary_logloss: 0.160922	valid's auc: 0.737744	valid's binary_logloss: 0.170897
[2203]	train's auc: 0.798339	train's binary_logloss: 0.160919	valid's auc: 0.737746	valid's binary_logloss: 0.170896
[2204]	train's auc: 0.798355	train's binary_logloss: 0.160916	valid's auc: 0.737739	valid's binary_logloss: 0.170897
[2205]	train's auc: 0.798383	train's binary_logloss: 0.160912	valid's auc: 0.737752	valid's binary_logloss: 0.170895
[2206]	train's auc: 0.798407	train's binary_logloss: 0.160907	valid's auc: 0.737752	valid's binary_logloss: 0.170895
[2207]	train's auc: 0.798422	train's binary_logloss: 0.160903	valid

LGBMClassifier(n_estimators=10000, silent=False, subsample_freq=1)

In [11]:
feat_imp = pd.Series(clf.feature_importances_)
feat_imp = feat_imp[feat_imp > 0]

imp_col_lst = list(feat_imp.sort_values(ascending=False).index)

pd.Series(imp_col_lst).to_csv(data_prep_dir + 'feature_importance.csv',index=False)

print('There are {} features with importance > 0.'.format(len(imp_col_lst)))

There are 11762 features with importance > 0.


In [12]:
n = clf.best_iteration_
base_loss = clf.best_score_['valid']['binary_logloss']
base_auc = clf.best_score_['valid']['auc']
print('Base loss is', base_loss, 'and AUC is', base_auc)

Base loss is 0.17089956808501636 and AUC is 0.7377981587205131


In [13]:
clf = LGBMClassifier(boosting_type='gbdt',
                     num_leaves=31, max_depth=-1, 
                     learning_rate=0.1, n_estimators=n, 
                     subsample_for_bin=200000, objective=None,
                     class_weight=None, min_split_gain=0.0, 
                     min_child_weight=0.001,
                     min_child_samples=20, subsample=1.0, subsample_freq=1,
                     colsample_bytree=1.0,
                     reg_alpha=0.0, reg_lambda=0.0, random_state=None,
                     n_jobs=-1, silent=False)

In [14]:
def valid_loss(cols):
    
    print('Running...')
    
    t0 = time.time()
    # select subset of features
    clf.fit(x_train[:, cols], y_train)
    
    y_pred = clf.predict_proba(x_valid[:, cols])[:, 1]
    
    t1 = time.time()
    print(t1 - t0, "s")
    
    return roc_auc_score(y_valid, y_pred)

all_num = int(len(feat_imp) / 100) * 100
print('There are', all_num, 'features waiting to be calculated')

loss = []
break_num = 0

print('Begin to feature selection...')
for i in range(500, all_num, 100):
    loss.append(valid_loss(imp_col_lst[:i]))
    if loss[-1] > base_auc:
        best_num = i
        base_auc = loss[-1]
        break_num += 1
    print('前', i, '个特征的得分为', loss[-1], '而全量得分', base_auc)
    print('\n')
    if break_num == 2:
        
        break
        
print('筛选出来最佳特征个数为', best_num, '这下子训练速度终于可以大大提升了')

There are 11700 features waiting to be calculated
Begin to feature selection...
Running...
[LightGBM] [Info] Number of positive: 337465, number of negative: 6701375
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 3.889155 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1156
[LightGBM] [Info] Number of data points in the train set: 7038840, number of used features: 500
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.047943 -> initscore=-2.988606
[LightGBM] [Info] Start training from score -2.988606
1739.7665176391602 s
前 500 个特征的得分为 0.7358498342854578 而全量得分 0.7377981587205131


Running...
[LightGBM] [Info] Number of positive: 337465, number of negative: 6701375
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 4.479049 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memo

## 保存被选择特征集

In [15]:
train_part_x = pd.DataFrame()
evals_x = pd.DataFrame()

# 重新读取数据集, 避免保存错了
feat_fname_train = ['train_x_sparse_onehot.npz', 'train_x_sparse_seq_len.npz', 'train_x_sparse_cntv.npz']
feat_fname_valid = ['valid_x_sparse_onehot.npz', 'valid_x_sparse_seq_len.npz', 'valid_x_sparse_cntv.npz']
x_train = pd.DataFrame()
x_valid = pd.DataFrame()
for tr, vl in zip(feat_fname_train, feat_fname_valid):
    train_f = sparse.load_npz(data_prep_dir + tr).tocsr()
    valid_f = sparse.load_npz(data_prep_dir + vl).tocsr()
    
    x_train = sparse.hstack((x_train, train_f)).tocsc()
    x_valid = sparse.hstack((x_valid, valid_f)).tocsc()
    print(tr, train_f.shape)
    print(vl, valid_f.shape)
    
print('x_train:', x_train.shape)
print('x_valid:', x_valid.shape)

print('Loading & merging features completed!')

train_x_sparse_onehot.npz (7038840, 1526)
valid_x_sparse_onehot.npz (1759974, 1526)
train_x_sparse_seq_len.npz (7038840, 8)
valid_x_sparse_seq_len.npz (1759974, 8)
train_x_sparse_cntv.npz (7038840, 323142)
valid_x_sparse_cntv.npz (1759974, 323142)
x_train: (7038840, 324676)
x_valid: (1759974, 324676)
Loading & merging features completed!


In [17]:
col2save = imp_col_lst[: best_num]
x_train2save = x_train[:, col2save]
x_valid2save = x_valid[:, col2save]

sparse.save_npz(data_prep_dir + 'train_x_sparse_selection.npz', x_train2save)
sparse.save_npz(data_prep_dir + 'valid_x_sparse_selection.npz', x_valid2save)

print('train_x_sparse_selection.npz', x_train2save.shape)
print('valid_x_sparse_selection.npz', x_valid2save.shape)


train_x_sparse_selection.npz (7038840, 800)
valid_x_sparse_selection.npz (1759974, 800)


# 模型构建

In [26]:
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=40, max_depth=-1, learning_rate=0.1, 
                n_estimators=10000, subsample_for_bin=200000, objective=None, 
                class_weight=None, min_split_gain=0.0, min_child_weight=0.001, 
                min_child_samples=20, subsample=0.7, subsample_freq=1, 
                colsample_bytree=0.7, 
                reg_alpha=6, reg_lambda=3,
                random_state=2018, n_jobs=-1, silent=False)

In [27]:
clf.fit(x_train2save, y_train, eval_set = [(x_train2save, y_train), (x_valid2save, y_valid)], 
        eval_names =['train','valid'],
        eval_metric='auc', early_stopping_rounds=50)

auc = int(clf.best_score_['valid']['auc']*1000000)
train_part_x = []
train_part_y = []

[LightGBM] [Info] Number of positive: 337465, number of negative: 6701375
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 6.629200 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1756
[LightGBM] [Info] Number of data points in the train set: 7038840, number of used features: 800
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.047943 -> initscore=-2.988606
[LightGBM] [Info] Start training from score -2.988606
[1]	train's auc: 0.641229	train's binary_logloss: 0.189886	valid's auc: 0.640498	valid's binary_logloss: 0.190082
Training until validation scores don't improve for 50 rounds
[2]	train's auc: 0.66071	train's binary_logloss: 0.188155	valid's auc: 0.659761	valid's binary_logloss: 0.18835
[3]	train's auc: 0.66179	train's binary_logloss: 0.186925	valid's auc: 0.660796	valid's binary_logloss: 0.187111
[4]	train's auc: 0.66314	train's 

[68]	train's auc: 0.714034	train's binary_logloss: 0.175509	valid's auc: 0.712814	valid's binary_logloss: 0.175812
[69]	train's auc: 0.714337	train's binary_logloss: 0.17546	valid's auc: 0.71313	valid's binary_logloss: 0.175761
[70]	train's auc: 0.714535	train's binary_logloss: 0.175419	valid's auc: 0.713293	valid's binary_logloss: 0.175723
[71]	train's auc: 0.714837	train's binary_logloss: 0.175371	valid's auc: 0.713602	valid's binary_logloss: 0.175674
[72]	train's auc: 0.715156	train's binary_logloss: 0.175324	valid's auc: 0.713886	valid's binary_logloss: 0.175632
[73]	train's auc: 0.715352	train's binary_logloss: 0.175281	valid's auc: 0.714078	valid's binary_logloss: 0.175592
[74]	train's auc: 0.715577	train's binary_logloss: 0.175242	valid's auc: 0.714337	valid's binary_logloss: 0.17555
[75]	train's auc: 0.715742	train's binary_logloss: 0.175195	valid's auc: 0.714506	valid's binary_logloss: 0.175504
[76]	train's auc: 0.715926	train's binary_logloss: 0.175155	valid's auc: 0.714661	v

[140]	train's auc: 0.726183	train's binary_logloss: 0.173346	valid's auc: 0.723904	valid's binary_logloss: 0.173811
[141]	train's auc: 0.726313	train's binary_logloss: 0.173326	valid's auc: 0.724011	valid's binary_logloss: 0.173795
[142]	train's auc: 0.726398	train's binary_logloss: 0.173305	valid's auc: 0.724082	valid's binary_logloss: 0.173777
[143]	train's auc: 0.726488	train's binary_logloss: 0.173289	valid's auc: 0.724174	valid's binary_logloss: 0.173762
[144]	train's auc: 0.726614	train's binary_logloss: 0.173271	valid's auc: 0.724279	valid's binary_logloss: 0.173747
[145]	train's auc: 0.726688	train's binary_logloss: 0.173255	valid's auc: 0.724336	valid's binary_logloss: 0.173734
[146]	train's auc: 0.72681	train's binary_logloss: 0.173234	valid's auc: 0.724478	valid's binary_logloss: 0.173713
[147]	train's auc: 0.726885	train's binary_logloss: 0.173218	valid's auc: 0.724544	valid's binary_logloss: 0.173699
[148]	train's auc: 0.726997	train's binary_logloss: 0.173201	valid's auc:

[211]	train's auc: 0.732173	train's binary_logloss: 0.172281	valid's auc: 0.729009	valid's binary_logloss: 0.172913
[212]	train's auc: 0.73234	train's binary_logloss: 0.172257	valid's auc: 0.72915	valid's binary_logloss: 0.172892
[213]	train's auc: 0.732401	train's binary_logloss: 0.172245	valid's auc: 0.729218	valid's binary_logloss: 0.172881
[214]	train's auc: 0.732513	train's binary_logloss: 0.17223	valid's auc: 0.729312	valid's binary_logloss: 0.172869
[215]	train's auc: 0.732667	train's binary_logloss: 0.172212	valid's auc: 0.729456	valid's binary_logloss: 0.172853
[216]	train's auc: 0.732763	train's binary_logloss: 0.172197	valid's auc: 0.729528	valid's binary_logloss: 0.172841
[217]	train's auc: 0.732819	train's binary_logloss: 0.172187	valid's auc: 0.729564	valid's binary_logloss: 0.172834
[218]	train's auc: 0.732921	train's binary_logloss: 0.172172	valid's auc: 0.729643	valid's binary_logloss: 0.172821
[219]	train's auc: 0.732985	train's binary_logloss: 0.172158	valid's auc: 0

[282]	train's auc: 0.736672	train's binary_logloss: 0.171521	valid's auc: 0.732414	valid's binary_logloss: 0.172329
[283]	train's auc: 0.736717	train's binary_logloss: 0.171514	valid's auc: 0.73244	valid's binary_logloss: 0.172323
[284]	train's auc: 0.736756	train's binary_logloss: 0.171507	valid's auc: 0.732462	valid's binary_logloss: 0.17232
[285]	train's auc: 0.736878	train's binary_logloss: 0.171491	valid's auc: 0.73257	valid's binary_logloss: 0.172306
[286]	train's auc: 0.736942	train's binary_logloss: 0.171482	valid's auc: 0.732607	valid's binary_logloss: 0.172298
[287]	train's auc: 0.736989	train's binary_logloss: 0.171473	valid's auc: 0.732643	valid's binary_logloss: 0.172292
[288]	train's auc: 0.737044	train's binary_logloss: 0.171466	valid's auc: 0.732669	valid's binary_logloss: 0.172288
[289]	train's auc: 0.737085	train's binary_logloss: 0.171459	valid's auc: 0.732712	valid's binary_logloss: 0.172282
[290]	train's auc: 0.737121	train's binary_logloss: 0.171452	valid's auc: 0

...

[1844]	train's auc: 0.771813	train's binary_logloss: 0.165438	valid's auc: 0.743682	valid's binary_logloss: 0.170231
[1845]	train's auc: 0.771819	train's binary_logloss: 0.165435	valid's auc: 0.743685	valid's binary_logloss: 0.17023
[1846]	train's auc: 0.771832	train's binary_logloss: 0.165433	valid's auc: 0.743688	valid's binary_logloss: 0.17023
[1847]	train's auc: 0.771842	train's binary_logloss: 0.165431	valid's auc: 0.743688	valid's binary_logloss: 0.17023
[1848]	train's auc: 0.771854	train's binary_logloss: 0.165428	valid's auc: 0.743697	valid's binary_logloss: 0.170229
[1849]	train's auc: 0.771869	train's binary_logloss: 0.165425	valid's auc: 0.743696	valid's binary_logloss: 0.170229
[1850]	train's auc: 0.771888	train's binary_logloss: 0.165422	valid's auc: 0.743697	valid's binary_logloss: 0.170229
[1851]	train's auc: 0.771902	train's binary_logloss: 0.16542	valid's auc: 0.743696	valid's binary_logloss: 0.170229
[1852]	train's auc: 0.771917	train's binary_logloss: 0.165418	valid'

[1915]	train's auc: 0.772879	train's binary_logloss: 0.165249	valid's auc: 0.743736	valid's binary_logloss: 0.170219
[1916]	train's auc: 0.772896	train's binary_logloss: 0.165246	valid's auc: 0.743737	valid's binary_logloss: 0.170218
[1917]	train's auc: 0.772909	train's binary_logloss: 0.165244	valid's auc: 0.743733	valid's binary_logloss: 0.170219
[1918]	train's auc: 0.772923	train's binary_logloss: 0.16524	valid's auc: 0.743735	valid's binary_logloss: 0.170218
[1919]	train's auc: 0.77295	train's binary_logloss: 0.165237	valid's auc: 0.743741	valid's binary_logloss: 0.170217
[1920]	train's auc: 0.772966	train's binary_logloss: 0.165235	valid's auc: 0.743741	valid's binary_logloss: 0.170217
[1921]	train's auc: 0.772978	train's binary_logloss: 0.165232	valid's auc: 0.743741	valid's binary_logloss: 0.170217
[1922]	train's auc: 0.772987	train's binary_logloss: 0.16523	valid's auc: 0.743738	valid's binary_logloss: 0.170217
[1923]	train's auc: 0.773002	train's binary_logloss: 0.165227	valid

[1986]	train's auc: 0.773895	train's binary_logloss: 0.165058	valid's auc: 0.743801	valid's binary_logloss: 0.170204
[1987]	train's auc: 0.773905	train's binary_logloss: 0.165055	valid's auc: 0.743801	valid's binary_logloss: 0.170203
[1988]	train's auc: 0.773915	train's binary_logloss: 0.165053	valid's auc: 0.743799	valid's binary_logloss: 0.170203
[1989]	train's auc: 0.773939	train's binary_logloss: 0.16505	valid's auc: 0.74379	valid's binary_logloss: 0.170204
[1990]	train's auc: 0.773945	train's binary_logloss: 0.165047	valid's auc: 0.743791	valid's binary_logloss: 0.170204
[1991]	train's auc: 0.773966	train's binary_logloss: 0.165043	valid's auc: 0.743794	valid's binary_logloss: 0.170203
[1992]	train's auc: 0.773983	train's binary_logloss: 0.16504	valid's auc: 0.743795	valid's binary_logloss: 0.170203
[1993]	train's auc: 0.773999	train's binary_logloss: 0.165038	valid's auc: 0.743798	valid's binary_logloss: 0.170203
[1994]	train's auc: 0.774012	train's binary_logloss: 0.165035	valid

[2057]	train's auc: 0.77491	train's binary_logloss: 0.164867	valid's auc: 0.743801	valid's binary_logloss: 0.170196
[2058]	train's auc: 0.774924	train's binary_logloss: 0.164864	valid's auc: 0.743797	valid's binary_logloss: 0.170197
[2059]	train's auc: 0.774933	train's binary_logloss: 0.164862	valid's auc: 0.7438	valid's binary_logloss: 0.170196
[2060]	train's auc: 0.77495	train's binary_logloss: 0.164859	valid's auc: 0.743797	valid's binary_logloss: 0.170197
[2061]	train's auc: 0.774967	train's binary_logloss: 0.164857	valid's auc: 0.743799	valid's binary_logloss: 0.170197
[2062]	train's auc: 0.774984	train's binary_logloss: 0.164854	valid's auc: 0.743803	valid's binary_logloss: 0.170197
[2063]	train's auc: 0.774995	train's binary_logloss: 0.164852	valid's auc: 0.743803	valid's binary_logloss: 0.170197
[2064]	train's auc: 0.775002	train's binary_logloss: 0.164849	valid's auc: 0.743805	valid's binary_logloss: 0.170196
[2065]	train's auc: 0.775021	train's binary_logloss: 0.164847	valid'

[2128]	train's auc: 0.775937	train's binary_logloss: 0.164684	valid's auc: 0.743805	valid's binary_logloss: 0.170191
[2129]	train's auc: 0.77595	train's binary_logloss: 0.164681	valid's auc: 0.743805	valid's binary_logloss: 0.17019
[2130]	train's auc: 0.775965	train's binary_logloss: 0.164678	valid's auc: 0.743807	valid's binary_logloss: 0.17019
[2131]	train's auc: 0.775984	train's binary_logloss: 0.164675	valid's auc: 0.743808	valid's binary_logloss: 0.17019
[2132]	train's auc: 0.776001	train's binary_logloss: 0.164673	valid's auc: 0.743807	valid's binary_logloss: 0.17019
[2133]	train's auc: 0.77602	train's binary_logloss: 0.16467	valid's auc: 0.743807	valid's binary_logloss: 0.170189
[2134]	train's auc: 0.776035	train's binary_logloss: 0.164667	valid's auc: 0.743801	valid's binary_logloss: 0.17019
[2135]	train's auc: 0.77605	train's binary_logloss: 0.164665	valid's auc: 0.743804	valid's binary_logloss: 0.17019
[2136]	train's auc: 0.776073	train's binary_logloss: 0.164661	valid's auc:

In [32]:
from sklearn.metrics import roc_auc_score, log_loss
print('===================Test Set Performance===================')
y_valid_pred = clf.predict_proba(x_valid2save, num_iteration = clf.best_iteration_)[:,1]
print(pd.Series(y_valid_pred).describe())

# round(pd.Series(evals_ypre),6).to_csv('data_preprocessing/evals_ypre_'+str(i+1)+'.csv',index=False)
print('Test AUC', roc_auc_score(y_valid, y_valid_pred))
print('Test Logloss', log_loss(y_valid, y_valid_pred))

===================Test Set Performance===================
count    1.759974e+06
mean     4.796217e-02
std      5.839436e-02
min      2.186377e-04
25%      2.071137e-02
50%      3.481173e-02
75%      5.510371e-02
max      9.953930e-01
dtype: float64
Test AUC 0.7438440879170812
Test Logloss 0.17018985844814913


# 组合ID特征

In [20]:
label_feat_lst = ['aid', 'advertiserId', 'campaignId', 'creativeId',
                  'creativeSize', 'adCategoryId', 'productId', 'productType', 'age',
                  'gender','education', 'consumptionAbility', 'LBS',
                  'os', 'carrier', 'house']

In [22]:
df_data = df_data[label_feat_lst]

for feature in tqdm(label_feat_lst):
    try:
        df_data[feature] = LabelEncoder().fit_transform(df_data[feature].apply(int))
    except:
        df_data[feature] = LabelEncoder().fit_transform(df_data[feature])


100%|██████████| 16/16 [00:56<00:00,  3.51s/it]


In [23]:
train_part_x_sparse = pd.DataFrame()
evals_x_sparse = pd.DataFrame()

feat_len = len(label_feat_lst)
enc = OneHotEncoder()
num = 0
for i in range(feat_len):
    for j in range(feat_len - i - 1):
        s = time.time()
        # 握手问题, 每一个feat都要和其他feat计算一次
        print(label_feat_lst[i], label_feat_lst[i + j + 1])
#         se = df_data[label_feat_lst[i]] * 100000 + df_data[label_feat_lst[i + j + 1]] * 1
#         enc.fit(se.values.reshape(-1, 1))
        
#         se = data.loc[train_part_index][col[i]]*100000+data.loc[train_part_index][col[i+j+1]]*1
#         arr =enc.transform(se.values.reshape(-1, 1))
#         train_part_x_sparse = sparse.hstack((train_part_x_sparse, arr))

        
#         se = data.loc[evals_index][col[i]]*100000+data.loc[evals_index][col[i+j+1]]*1
#         arr = enc.transform(se.values.reshape(-1, 1))
#         evals_x_sparse = sparse.hstack((evals_x_sparse,arr))

#         num+=1
#         arr = []
#         print(num,col[i],col[i+j+1],int(time.time()-s),"s")
#         if num % 12==0:
#             k = num//12
#             ##存写组合稀疏矩阵
#             print("Saving...")
#             print(k)
#             print('train_part_x...')
#             sparse.save_npz('data_preprocessing/train_part_x_sparse_two_'+str(k)+'.npz',train_part_x_sparse)
#             print('evals_x...')
#             sparse.save_npz('data_preprocessing/evals_x_sparse_two_'+str(k)+'.npz',evals_x_sparse)
#             print('test1_x...')
#             sparse.save_npz('data_preprocessing/test1_x_sparse_two_'+str(k)+'.npz',test1_x_sparse)
#             print('test2_x...')
#             sparse.save_npz('data_preprocessing/test2_x_sparse_two_'+str(k)+'.npz',test2_x_sparse)
#             print('Over')
#             train_part_x_sparse=pd.DataFrame()
#             evals_x_sparse=pd.DataFrame()
#             test1_x_sparse=pd.DataFrame()
#             test2_x_sparse=pd.DataFrame()

aid advertiserId
aid campaignId
aid creativeId
aid creativeSize
aid adCategoryId
aid productId
aid productType
aid age
aid gender
aid education
aid consumptionAbility
aid LBS
aid os
aid carrier
aid house
advertiserId campaignId
advertiserId creativeId
advertiserId creativeSize
advertiserId adCategoryId
advertiserId productId
advertiserId productType
advertiserId age
advertiserId gender
advertiserId education
advertiserId consumptionAbility
advertiserId LBS
advertiserId os
advertiserId carrier
advertiserId house
campaignId creativeId
campaignId creativeSize
campaignId adCategoryId
campaignId productId
campaignId productType
campaignId age
campaignId gender
campaignId education
campaignId consumptionAbility
campaignId LBS
campaignId os
campaignId carrier
campaignId house
creativeId creativeSize
creativeId adCategoryId
creativeId productId
creativeId productType
creativeId age
creativeId gender
creativeId education
creativeId consumptionAbility
creativeId LBS
creativeId os
creativeId carr

# 模型构建

In [14]:
def model_evaluation(y_train, y_train_pred, y_test, y_test_pred, model_name=''):
    """

    :param y_train:
    :type y_train:
    :param y_train_pred:
    :type y_train_pred:
    :param y_test:
    :type y_test:
    :param y_test_pred:
    :type y_test_pred:
    :param model_name:
    :type model_name:
    :return:
    :rtype:
    """
    print("{} Train AUC: {}, logloss: {}".format(model_name, roc_auc_score(y_train, y_train_pred), log_loss(y_train, y_train_pred)))
    # print("{} Train confusion matrix: {}".format(model_name, confusion_matrix(y_train, y_train_pred)))

    print("{} Test AUC: {}, logloss: {}".format(model_name, roc_auc_score(y_test, y_test_pred), log_loss(y_test, y_test_pred)))
    # print("{} Test confusion matrix: {}".format(model_name, confusion_matrix(y_test, y_test_pred)))

## Baseline版本
- 只使用离散特征，进行onehot编码
- 线性模型：LogisticRegression

In [13]:
print(x_train.shape, x_valid.shape)
# baseline版本
turned_param = {
        'penalty': 'l2',
        'C': 10,
        'solver': 'lbfgs',
        'tol': 1e-4,
        'max_iter': 10000
    }

clf = LogisticRegression(random_state=1, **turned_param)
clf.fit(x_train, y_train)

y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_valid)

model_evaluation(y_train, y_train_pred, y_valid, y_test_pred, model_name='LogisticRegression')


(7038840, 1526) (1759974, 1526)


NameError: name 'model_evaluation' is not defined

In [15]:
# 基于相似度


LogisticRegression Train AUC: 0.5000014816351326, logloss: 1.6558969428673103
LogisticRegression Test AUC: 0.5, logloss: 1.6581997519647336


In [15]:
for i, j in enumerate([1,1,1]):
    print(i, j)

0 1
1 1
2 1
